## 1. 환경 설정

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [4]:
# seed 고정하기
torch.manual_seed(42)

# gpu 사용하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 2. 데이터 준비

In [5]:
# 이미지 변형을 위한 클래스
transform = transforms.Compose([transforms.ToTensor(),  # 데이터를 텐서로 바꿔주기
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 정규화하는 코드, Normalize(mean, std)이다. 괄호 안에 3개는 RGB의 3채널을 의미한다.
                                ])

# CIFAR-10 dataset 불러오기
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 학습, 검증 데이터셋 9:1 비율로 분리
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(trainset, [train_size, val_size])

# dataloader 생성하기
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

100%|██████████| 170498071/170498071 [00:04<00:00, 42520641.69it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 3. 모델 정의

In [6]:
class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv = nn.Conv2d(3, 2, kernel_size=5, stride=1, padding=2) # Conv2d(intput 채널, ouput 채널)
        self.pool = nn.MaxPool2d(2, 2) # MaxPool2d(커널 사이즈, 스트라이드)
        self.conv1 = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 2, kernel_size=3, stride=1, padding=1)
        self.fc0 = nn.Linear(4 * 4 * 2, 256) # 32 x 32는 해상도를 의미하고, 2는 채널 수를 의미함
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv(x)))
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 4 * 4 * 2) # view()는 reshape하는 함수
        x = torch.relu(self.fc0(x))
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2) # Conv2d(intput 채널, ouput 채널)
        self.pool = nn.MaxPool2d(2, 2) # MaxPool2d(커널 사이즈, 스트라이드)
        self.conv1 = nn.Conv2d(32, 24, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(24, 16, kernel_size=3, stride=1, padding=1)
        self.fc0 = nn.Linear(4 * 4 * 16, 256) # 4 x 4는 해상도를 의미하고, 16는 채널 수를 의미함
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv(x)))
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 4 * 4 * 16) # view()는 reshape하는 함수, 요기서 -1은 배치 크기만큼 알아서 reshape하라는 의미를 갖음.
        x = torch.relu(self.fc0(x))
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 4. 학습과 추론

In [33]:
# 모델 생성, loss function과 optimizer 정의하기
model1 = Model1().to(device)
model2 = Model2().to(device)

criterion = nn.CrossEntropyLoss() # cross entropy loss 사용하기

optimizer1 = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)
optimizer2 = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)

total_epoch = 10
iteration = len(trainloader)

In [15]:
# model1 학습
print('Model1 학습 시작')
for epoch in range(total_epoch):  # adjust the number of epochs as needed
    training_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer1.zero_grad() # 항상 맨 앞에 넣어줘서 gradient값들 초기화하기
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer1.step()
        training_loss += loss.item()
    training_loss = training_loss / iteration
    print(f'epoch: {epoch + 1:2d} loss: {training_loss:.3f}')

print('Model1 학습 완료')



Model1 학습 시작


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch:  1 loss: 2.301
epoch:  2 loss: 2.076
epoch:  3 loss: 1.759
epoch:  4 loss: 1.647
epoch:  5 loss: 1.593
epoch:  6 loss: 1.544
epoch:  7 loss: 1.509
epoch:  8 loss: 1.483
epoch:  9 loss: 1.454
epoch: 10 loss: 1.435
Model1 학습 완료


In [34]:
# model2 학습
print('Model2 학습 시작')
for epoch in range(total_epoch):  # adjust the number of epochs as needed
    training_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer2.zero_grad() # 항상 맨 앞에 넣어줘서 gradient값들 초기화하기
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer2.step()
        training_loss += loss.item()
    training_loss = training_loss / iteration
    print(f'epoch: {epoch + 1:2d} loss: {training_loss:.3f}')

print('Model2 학습 완료')



Model2 학습 시작
epoch:  1 loss: 2.196
epoch:  2 loss: 1.685
epoch:  3 loss: 1.437
epoch:  4 loss: 1.284
epoch:  5 loss: 1.171
epoch:  6 loss: 1.079
epoch:  7 loss: 1.003
epoch:  8 loss: 0.933
epoch:  9 loss: 0.876
epoch: 10 loss: 0.825
Model2 학습 완료


In [27]:
# model1 추론
model1.eval()
misclassified_examples1 = []
misclassified_classs1 = [0] * 10

with torch.no_grad():
    for idx, data in enumerate(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model1(inputs)
        _, predicted = torch.max(outputs, 1)
        label = labels.item()
        if predicted != label: # 예측에 실패한 경우
            misclassified_examples1.append(idx) # 예측 실패 idx 저장
            misclassified_classs1[label] += 1
    test_acc = (len(testloader) - len(misclassified_examples1))/len(testloader)*100
    print(f"Test Accuracy: {test_acc:.2f}%")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Test Accuracy: 47.62%


In [23]:
# model2 추론
model2.eval()
misclassified_examples2 = []
misclassified_classs2 = [0] * 10

with torch.no_grad():
    for idx, data in enumerate(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model2(inputs)
        _, predicted = torch.max(outputs, 1)
        label = labels.item()
        if predicted != label: # 예측에 실패한 경우
            misclassified_examples2.append(idx) # 예측 실패 idx 저장
            misclassified_classs2[label] += 1
    test_acc = (len(testloader) - len(misclassified_examples2))/len(testloader)*100
    print(f"Test Accuracy: {test_acc:.2f}%")

Test Accuracy: 68.47%


## 5. 결과 분석

In [31]:
count = 0

# Model 1 분류 잘못한 예제 번호 출력
for i in range(len(misclassified_examples1)):
    print(f'{misclassified_examples1[i]} ', end='')
    if count % 10 == 0:
        print()
    count += 1

# 분류 잘못한 총 개수 출력
print(f'\nNumber of Misclassified Examples: {len(misclassified_examples1)}')

# 각 클랫스의 잘못 분류한 갯수 출력
print('Model2: 각 클래스의 분류 오류 개수')
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for i in range(10):
    print(f'({i}) {classes[i]}: {misclassified_classs1[i]}개')

7 
10 12 15 17 22 24 25 27 28 31 
33 35 37 40 42 43 46 47 48 49 
52 53 56 57 59 61 63 65 67 68 
70 71 72 74 75 78 82 85 86 87 
91 92 95 96 97 98 99 101 102 103 
105 106 108 109 110 112 114 115 118 120 
121 125 126 127 128 131 132 134 135 136 
137 138 139 140 143 144 145 147 148 149 
150 153 154 155 158 159 160 162 164 165 
167 168 169 172 176 177 178 179 180 181 
182 183 184 188 189 190 192 194 195 197 
198 201 202 203 205 207 209 210 211 212 
213 214 215 218 219 221 223 224 227 228 
230 232 233 236 237 238 239 242 245 246 
247 249 250 254 255 257 258 259 260 262 
263 264 266 267 269 270 273 275 276 277 
279 280 287 291 294 302 303 305 306 307 
309 312 313 314 318 319 320 322 323 324 
325 326 327 330 332 337 340 342 343 349 
350 351 352 354 356 357 358 360 361 363 
365 366 367 368 370 374 376 377 378 380 
383 385 386 387 388 389 390 391 393 396 
397 399 400 401 404 405 407 409 411 412 
413 416 418 420 421 423 424 426 428 429 
430 431 432 434 436 437 438 439 443 444 
445 446 448 450 453

In [30]:
count = 0

# Model 2 분류 잘못한 예제 번호 출력
for i in range(len(misclassified_examples2)):
    print(f'{misclassified_examples2[i]} ', end='')
    if count % 10 == 0:
        print()
    count += 1

# 분류 잘못한 총 개수 출력
print(f'\nNumber of Misclassified Examples: {len(misclassified_examples2)}')

# 각 클랫스의 잘못 분류한 갯수 출력
print('Model2: 각 클래스의 분류 오류 개수')
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for i in range(10):
    print(f'({i}) {classes[i]}: {misclassified_classs2[i]}개')

0 
6 7 20 21 22 24 25 27 30 32 
33 35 37 40 46 50 52 57 58 59 
61 63 64 66 71 76 77 85 87 91 
99 106 112 122 126 127 128 129 138 139 
140 143 147 149 158 160 162 164 168 171 
172 176 188 193 195 198 201 210 211 216 
219 221 223 224 226 227 228 229 232 233 
242 245 247 249 254 256 258 262 263 264 
269 273 275 277 278 279 301 302 304 306 
309 312 313 314 321 322 323 324 327 328 
335 340 341 343 344 346 352 354 356 363 
365 366 368 376 377 378 383 384 385 388 
396 398 399 401 411 414 417 418 421 422 
426 428 433 434 436 439 441 443 444 449 
450 453 455 456 458 459 460 463 464 465 
466 468 474 477 480 485 488 494 503 513 
515 518 525 526 528 529 530 531 532 538 
541 548 549 551 555 557 563 565 567 568 
569 570 574 577 582 583 593 594 606 607 
616 621 628 630 631 637 639 641 652 653 
655 661 663 665 668 669 671 672 674 676 
677 678 680 683 685 689 690 697 701 702 
706 707 711 718 719 720 725 727 730 731 
733 734 735 737 740 746 749 751 760 761 
762 766 767 770 771 773 775 776 779 790 
792 7